In [1]:
import os
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv, find_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate   
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableLambda
from pydantic import BaseModel, Field

In [2]:
#! TODO - Instantiate your chat model
# * Carrega as variáveis de ambiente
_ = load_dotenv(find_dotenv())

# * Verifica se a API key está configurada
if not os.getenv("OPENAI_API_KEY"):
    raise ValueError("OPENAI_API_KEY não encontrada no arquivo .env")

# * Configura o modelo com parâmetros específicos
model: ChatOpenAI = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.0,  # Controla a criatividade das respostas
)  # type: ignore

response = model.invoke("The Sky is ?")
print(response.content)

blue


In [5]:
model = ChatOpenAI(model="gpt-4.1", temperature=0.0)
translator_template = ChatPromptTemplate.from_messages([
    ("system", "Você é um tradutor especializado. Traduza o texto para {idioma}."),
    ("human", "{texto}")
])
parser_output = StrOutputParser()

translator_chain = translator_template | model | parser_output

response = translator_chain.invoke({"texto": "Hello, how are you today?", "idioma": "português"})
print(response)

Olá, como você está hoje?


# Tutorial LangChain: Runnable, RunnableSequence, RunnableMap e Métodos

## 1. Conceito Fundamental: Runnable

O `Runnable` é a interface base do LangChain que padroniza como diferentes componentes podem ser executados e combinados. Qualquer objeto que implementa `Runnable` possui métodos como `.invoke()`, `.stream()`, `.batch()`, etc.

### Componentes Runnable no seu código:
```python
# Todos estes são objetos Runnable:
translator_template = ChatPromptTemplate.from_messages([...])  # Runnable
model = ChatOpenAI(model="gpt-4.1", temperature=0.0)          # Runnable  
parser_output = StrOutputParser()                             # Runnable
```

## 2. RunnableSequence (Operador Pipe `|`)

Quando você usa o operador `|`, está criando uma `RunnableSequence` - uma cadeia onde a saída de um componente vira entrada do próximo.

### Seu código cria esta sequência:
```python
translator_chain = translator_template | model | parser_output
# Isso é equivalente a:
# RunnableSequence(first=translator_template, middle=[model], last=parser_output)
```

### Fluxo de dados:
```
Input: {"texto": "Hello...", "idioma": "português"}
  ↓
translator_template → Gera prompt formatado
  ↓
model → Processa prompt e gera resposta
  ↓
parser_output → Extrai string da resposta
  ↓
Output: "Olá, como você está hoje?"
```

## 3. Método .invoke()

O `.invoke()` executa a cadeia de forma síncrona com um único input:

```python
# Seu código usa invoke:
response = translator_chain.invoke({
    "texto": "Hello, how are you today?", 
    "idioma": "português"
})
```

### Outros métodos de execução:
```python
# Execução assíncrona
response = await translator_chain.ainvoke(input_data)

# Processamento em lote
responses = translator_chain.batch([input1, input2, input3])

# Streaming (útil para respostas longas)
for chunk in translator_chain.stream(input_data):
    print(chunk, end="")
```

## 4. Método .pipe()

O `.pipe()` permite adicionar mais componentes à cadeia existente:

```python
# Criando uma cadeia base
base_chain = translator_template | model

# Adicionando parser com pipe
complete_chain = base_chain.pipe(parser_output)

# Ou adicionando mais processamento
def capitalize_output(text):
    return text.upper()

enhanced_chain = translator_chain.pipe(capitalize_output)
```

## 5. Método .bind()

O `.bind()` permite "fixar" parâmetros em um Runnable:

```python
# Exemplo com seu modelo
model_with_params = model.bind(
    temperature=0.7,
    max_tokens=100
)

# Criando template específico para francês
french_translator = translator_template.bind(idioma="francês")

# Agora só precisa passar o texto:
french_chain = french_translator | model | parser_output
response = french_chain.invoke({"texto": "Hello world"})
```

## 6. RunnableMap

O `RunnableMap` permite executar múltiplos Runnables em paralelo:

```python
from langchain_core.runnables import RunnableMap

# Criando tradutores para diferentes idiomas
spanish_chain = translator_template.bind(idioma="espanhol") | model | parser_output
french_chain = translator_template.bind(idioma="francês") | model | parser_output

# Executando em paralelo
multi_translator = RunnableMap({
    "spanish": spanish_chain,
    "french": french_chain,
    "portuguese": translator_chain
})

# Resultado será um dicionário com todas as traduções
results = multi_translator.invoke({"texto": "Hello world"})
# Output: {
#   "spanish": "Hola mundo",
#   "french": "Bonjour le monde", 
#   "portuguese": "Olá mundo"
# }
```

## 7. Exemplo Completo Expandido

```python
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableMap, RunnableLambda

# Componentes base
model = ChatOpenAI(model="gpt-4", temperature=0.0)
translator_template = ChatPromptTemplate.from_messages([
    ("system", "Você é um tradutor especializado. Traduza o texto para {idioma}."),
    ("human", "{texto}")
])
parser_output = StrOutputParser()

# Função para contar palavras
def count_words(text):
    return {"translation": text, "word_count": len(text.split())}

# Cadeia completa com processamento adicional
enhanced_chain = (
    translator_template 
    | model 
    | parser_output 
    | RunnableLambda(count_words)
)

# Múltiplas operações em paralelo
parallel_operations = RunnableMap({
    "translation": translator_chain,
    "original_length": RunnableLambda(lambda x: len(x["texto"].split())),
    "target_language": RunnableLambda(lambda x: x["idioma"])
})

# Execução
input_data = {"texto": "Hello, how are you today?", "idioma": "português"}
result = parallel_operations.invoke(input_data)
```

## 8. Vantagens da Arquitetura Runnable

1. **Composabilidade**: Fácil combinação de componentes
2. **Padronização**: Interface uniforme para todos os componentes
3. **Flexibilidade**: Métodos síncronos, assíncronos, batch e streaming
4. **Debugging**: Cada etapa pode ser testada independentemente
5. **Reutilização**: Componentes podem ser reutilizados em diferentes cadeias

## 9. Dicas Práticas

- Use `|` para sequências lineares
- Use `RunnableMap` para operações paralelas
- Use `.bind()` para fixar parâmetros frequentemente usados
- Use `.pipe()` para adicionar processamento a cadeias existentes
- Teste cada componente individualmente antes de combiná-los

### `RunnableParallel`

- Executa múltiplas operações SIMULTANEAMENTE
- cada operação recebe a mesma entrada
- O resultado é um dicionário com chaves nomeadas
- Ideal para operações independentes que podem rodar em paralelo

Dica: `RunnableParallel` é perfeito para qdo se precisa fazer múltiplas coisas ao mesmo tempo, como processar um texto e salvar logs, ou analisar diferentes aspectos de uma ideia simultaneamente.

In [6]:
logs = []

def log_and_pass(text: str) -> str:
    """Log simples que passa o texto adiante"""
    print(f"[LOG] Tradução gerada: {text}")
    
# Cadeia com log simples
model = ChatOpenAI(model="gpt-4.1", temperature=0.0)
translator_template = ChatPromptTemplate.from_messages(
    [
        ("system", "Você é um tradutor especializado. Traduza o texto para {idioma}."),
        ("human", "{texto}")
    ]
)
parser_output = StrOutputParser()

# Adicionando log após o parse
translator_chain_with_log = (
    translator_template
    | model
    | parser_output 
    | RunnableLambda(log_and_pass)
)

response = translator_chain.invoke({"texto": "Hello, how are you today?", "idioma": "português"})
print(response)

Olá, como você está hoje?


In [12]:
# Lista para guardar os logs
logs = []

# Cadeia com log simples
model = ChatOpenAI(model="gpt-4o", temperature=0.0)
translator_template = ChatPromptTemplate.from_messages(
    [
        ("system", "Você é um tradutor especializado. Traduza o texto para {idioma}."),
        ("human", "{texto}")
    ]
)

# Parser simples
parser = StrOutputParser()

# Chain que faz parse E salva log ao mesmo tempo
parser_and_log_output_chain = RunnableParallel(
    output=parser,
    log=RunnableLambda(lambda x: logs.append(x))
)

# Montando a chain completa
translator_chain = translator_template | model | parser_and_log_output_chain

# Testando a chain de tradução
print("=== TESTANDO CHAIN DE TRADUÇÃO COM LOG ===")

# Teste 1: Português para Inglês
resultado1 = translator_chain.invoke({
    "idioma": "inglês",
    "texto": "Olá, como você está hoje?"
})

print("Teste 1 - Português para Inglês:")
print(f"  Tradução: {resultado1['output']}")
print(f"  Logs salvos até agora: {len(logs)}")

print("\n" + "-"*50)

=== TESTANDO CHAIN DE TRADUÇÃO COM LOG ===
Teste 1 - Português para Inglês:
  Tradução: Hello, how are you today?
  Logs salvos até agora: 1

--------------------------------------------------


In [13]:
# Teste 2: Português para Francês
resultado2 = translator_chain.invoke({
    "idioma": "francês", 
    "texto": "Eu gosto muito de programar em Python"
})

print("Teste 2 - Português para Francês:")
print(f"  Tradução: {resultado2['output']}")
print(f"  Logs salvos até agora: {len(logs)}")

print("\n" + "-"*50)

Teste 2 - Português para Francês:
  Tradução: J'aime beaucoup programmer en Python.
  Logs salvos até agora: 2

--------------------------------------------------


In [14]:
# Teste 3: Português para Espanhol
resultado3 = translator_chain.invoke({
    "idioma": "espanhol",
    "texto": "A inteligência artificial está evoluindo rapidamente"
})

print("Teste 3 - Português para Espanhol:")
print(f"  Tradução: {resultado3['output']}")
print(f"  Logs salvos até agora: {len(logs)}")

print("\n" + "="*50)
print("TODOS OS LOGS CAPTURADOS:")
for i, log in enumerate(logs, 1):
    print(f"Log {i}: {log}")

print(f"\nTotal de traduções logadas: {len(logs)}")

Teste 3 - Português para Espanhol:
  Tradução: La inteligencia artificial está evolucionando rápidamente.
  Logs salvos até agora: 3

TODOS OS LOGS CAPTURADOS:
Log 1: content='Hello, how are you today?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 32, 'total_tokens': 39, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_a288987b44', 'id': 'chatcmpl-C0T4ER5RlHAO5iHkz6qHDEd64O6Ge', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--3483d7bd-c3fb-4750-a388-0e95dd992b6d-0' usage_metadata={'input_tokens': 32, 'output_tokens': 7, 'total_tokens': 39, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
Log 2: content="J'aime beaucou